In [ ]:
path_image_g = '../../../belen/masa/NGC1313_image_g.fits'
path_image_i = '../../../belen/masa/NGC1313_image_i.fits'
path_weight_g = '../../../belen/decals_data/ngc1313/ngc1313_weight_g.fits'
path_weight_i = '../../../belen/decals_data/ngc1313/ngc1313_weight_i.fits'